In [1]:
from transformers import ElectraModel, ElectraTokenizer
import torch
import numpy as np
import random
from transformers import pipeline
import json
from  tqdm import tqdm
import copy
import os


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
fill_mask = pipeline(
    "fill-mask",
    model="monologg/koelectra-base-v2-generator",
    tokenizer="monologg/koelectra-base-v2-generator",
    device = 0
)


In [4]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v2-generator")

In [5]:
def non_change_set():
    non_change_set = [0,1,2,3,4]
    symbols = [",","'",'"',"!","?",".","@","#","$","%","^","&","*","(",")","-","_","+","="]
    for symbol in symbols:
        non_change_set.append(tokenizer.encode([symbol])[1])
    return list(set(non_change_set))

In [6]:
non_change_ids = non_change_set()

In [7]:
def parse_text(contents, answer, ratio, non_change_ids):
    
    r = []
    
    inputs = fill_mask._parse_and_tokenize(contents)
    non_change_ids+=tokenizer.encode(answer)
    non_change_ids = list(set(non_change_ids))
    for content_input_ids in inputs["input_ids"]:
        can_change_index = content_input_ids!=-1
        for _id in non_change_ids:
            can_change_index = can_change_index & (content_input_ids!=_id) 
        can_change_index = can_change_index.nonzero()
        #print(can_change_index)
        try:
            random_masked_index = random.sample(range(0, can_change_index.shape[0]), int(can_change_index.shape[0]*ratio))
        except:
            return None
        #print(random_masked_index)
        for i in random_masked_index:
            #print(can_change_index[i])
            content_input_ids[can_change_index[i]]=fill_mask.tokenizer.mask_token_id
        #print(fill_mask.tokenizer.decode(content_input_ids, skip_special_tokens=True))
    
    for temp in inputs["input_ids"]:
        if temp.shape[0]>510:
            return 
    outputs = fill_mask._forward(inputs)
    
    
    batch_size = outputs.shape[0] 

    for i in range(batch_size):
        input_ids = inputs["input_ids"][i]
        result = []

        masked_index = (input_ids == fill_mask.tokenizer.mask_token_id).nonzero()
        #print(masked_index)
        
        for index in masked_index:
            tokens = input_ids.numpy()
            tokens[index] = np.argmax(outputs[i][index.item()])
        tokens = tokens[np.where(tokens != fill_mask.tokenizer.pad_token_id)]
        r.append(fill_mask.tokenizer.decode(tokens, skip_special_tokens=True))
    return r

In [30]:
with open("korquad_open_train.json", "r", encoding='utf-8') as reader:
    all_data = json.load(reader)
reader.close()

In [31]:
data = all_data['data']

In [32]:
c = 0
no_c = 0
for i, ele in tqdm(enumerate(data)):
    question = ele['qa']['question']
    answer = ele['qa']['answer']
    paragraphs = ele['paragraphs']
    new_ele = []
    check = []
    for paragraph in paragraphs:
        
        if paragraph['label'] == '1':
            
            c+=1
            #new_ele.append(paragraph)
            
            content = paragraph['contents']
            try:
                non_change_ids = non_change_set()
                new_contents = parse_text([content,content],answer, 0.25, non_change_ids)
            except:
                continue
            
            if new_contents == None:
                no_c+=1
                #print(no_c, i)
                continue
            
            for new_content in new_contents:
                new_paragraph = copy.deepcopy(paragraph)
                new_paragraph['contents'] = new_content
                new_ele.append(new_paragraph)
        else:
            continue
        
    
    ele['paragraphs'] += new_ele
    '''if c>1:
        print(ele['paragraphs'])
        break'''

60367it [2:24:09,  6.98it/s]


In [33]:
with open("korquad_open_train_aug_2_times.json", "w") as json_file:

    json.dump(all_data, json_file)


In [26]:
data[20000]

{'qa': {'question': '소연은 형이 살해당한 일을 계기로 누구를 타도하기위해 거병하였는가?', 'answer': '소보권'},
 'paragraphs': [{'title': '양_무제',
   'contents': '기록에는 그가 어려서부터 문무를 두루 통달하여 주목받았다고 적고 있다. 일찍이 남제 문화의 중심지였던 경릉왕(竟陵王) 소자량의 서저(西邸)에도 드나들었으며, 심약(沈約) 등과 함께 경릉팔우(竟陵八友)의 한 사람으로 수학하였다. 소연이 옹주자사(雍州刺史)가 되었을 때, 당시 남제의 황제였던 소보권이 한창 폭정을 휘두르고 있었다. 그는 소의를 비롯한 황족 종친뿐 아니라 많은 대신과 사람들을 죽였고 궁전에선 과도한 사치를 일삼았고, 소연은 형이 살해당한 일을 계기로 그를 타도하기 위하여 거병했다. 수도인 건강으로 진군해 소보권을 살해하고 화제를 옹립했으나, 이듬해인 502년에 화제로부터 제위를 선양받고 양나라를 세우게 되었다.',
   'label': '1',
   'source': 'korquad'},
  {'title': '양_무제',
   'contents': '기록에는 그가 어려서부터 문무를 두루 통달하여 주목받았다고 적고 있다. 소관제 문화의 중심지였던 소천왕 ( 王 ) 소자량의 서저 ( ) 에 드나들었으며, 심약 ( 沈 ) 등과 함께 경릉팔우 ( 八 ) 의 한 사람으로 수학하였다. 소연이 옹주자사 ( 州 史 ) 가 되었을 때, 당시 대제의 지도자였던 소보권은 한창 폭정을 휘두르고 있었다. 그는 소연의의의귀족 종친뿐 아니라 많은 대신과 사람들을 죽였으며, 관선에의 사치를 일삼았고, 소연은 자신이 살해당한 일을 일으켜하여제를 타도하기 위하여 거병했다. 수도의 건강을 행군해 소보권을 살해하고 화제를 옹립했으나, 이듬해인 502년에 화제로부터 제위를 선양받고 양위를 얻게 되었다.',
   'label': '1',
   'source': 'korquad'}]}

In [27]:
count = 0
#no_c = 0
count_2 = 0
for i, ele in tqdm(enumerate(all_data['data'])):
    question = ele['qa']['question']
    answer = ele['qa']['answer']
    paragraphs = ele['paragraphs']
    new_ele = []
    check = []
    for paragraph in paragraphs:
        
        if paragraph['label'] == '1':
            count+=1
        else:
            count_2+=1

120774it [00:00, 239162.14it/s]


In [29]:
count

354396